In [109]:
import os
import subprocess
import Bio
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
from Bio.Seq import Seq #added
import pandas as pd
import genomicranges as gr
from Bio.Seq import Seq
import pyranges as pr
from Bio.SeqUtils import GC
from Bio import AlignIO, SeqIO

In [2]:
dir(Bio.SeqUtils)

['Alphabet',
 'GC',
 'GC123',
 'GC_skew',
 'IUPACData',
 'MutableSeq',
 'Seq',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'cos',
 'molecular_weight',
 'nt_search',
 'pi',
 'print_function',
 're',
 'seq1',
 'seq3',
 'sin',
 'six_frame_translations',
 'xGC_skew']

## plan: 
    -need to do this for each human - species file individually (or else the alignment cols may be off) 
    -find regions of human intron that have the same GC content as the promoter (concatenated together) 
    -concatenate those together 

In [3]:
#make dictionary of species and GC content of their promoters - species: [x_prom, y_prom]

#do this analysis without the lemur or the loris

species= ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'marm', 'mac']
gc_prom_dict = {}

for spec in species: 
    sample_seqX = AlignIO.read('all_species/' + spec + '_promoter_msa_X.phy', "phylip")
    sample_seqY = AlignIO.read('all_species/' + spec + '_promoter_msa_Y.phy', "phylip")
    gc_prom_dict[spec] = [GC(sample_seqX[0].seq), GC(sample_seqY[0].seq)]
gc_prom_dict    

{'chimp': [62.19463356027233, 54.532403609515995],
 'gorilla': [63.55140186915888, 53.83930558646784],
 'orangutan': [63.34501752628943, 53.99230943225515],
 'pileatedgibbon': [64.09912926992632, 52.87979966611018],
 'marm': [62.563559322033896, 48.24835241068332],
 'mac': [62.5, 54.52469904524699]}

In [216]:

same_gc_introns = {} #keys are the species (of that particular file), [[1,2,3], [4,5,6]] #starts list and end list
                       #can make these files have the exact same GC content 
chroms = ['X', 'Y']


for ix in range(1,len(samples)): #only running through sample_seqY 
    
    chrom = chroms[ix]
    
    for spec in species:
        sample_seqX = AlignIO.read('all_species/' + spec + '_intron_msa_X.phy', "phylip")
        sample_seqY = AlignIO.read('all_species/' + spec + '_intron_msa_Y.phy', "phylip")
        samples = [sample_seqX, sample_seqY]
        
        sample = samples[ix]
        print(sample)
        print(spec)
        target = gc_prom_dict[spec][ix]
        range_gc = [target - 3, target + 3]
        #print(range_gc)
        for s in range(0, len(sample[0].seq), 500):
                                                                #take thousand intervals that are +/- 5% of the promoter GC
            e = s + 1000
            gc_perc = GC(sample[0][s:e].seq)
            if (range_gc[0] < gc_perc) & (gc_perc < range_gc[1]):
               # print('yes', spec, gc_perc, ix, chrom)
                try:
                    same_gc_introns[(spec,chrom)][0].append(s)
                    same_gc_introns[(spec,chrom)][1].append(e)
                except: 
                    same_gc_introns[(spec,chrom)] = [[s],[e]]


SingleLetterAlphabet() alignment with 2 rows and 578799 columns
TCAGAGAGAAAGAGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TGC human
TCAGAGAGAAAGTGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TGC sequence2
chimp
SingleLetterAlphabet() alignment with 2 rows and 517053 columns
TCAGAGAGAAAGAGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TCC human
TCAGAGAGAAAGAGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TCC sequence2
gorilla
SingleLetterAlphabet() alignment with 2 rows and 537580 columns
TCAGAGAGAAAGAGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TGC human
TCAGAGAGAAAGAGCCAGAGGAAAAGAGGGGGTAACTGAGTGAC...TGC sequence2
orangutan
SingleLetterAlphabet() alignment with 2 rows and 446581 columns
TGCTTTATTTTTTATCTTTATTTCTAGATTATTTAGATCTTTTT...TGC human
TGTTTTGTTTTCTTTGTTTGTTTTGTGGCTTTCTCGTTCCTTTG...TGC sequence2
pileatedgibbon
SingleLetterAlphabet() alignment with 2 rows and 222295 columns
AATAATGATTCCTTGGATATACCAAGTCTGGATAGCGGATTCGA...TCC human
AATAATAATTTCTAGGATATATTAAGTCTGGATATAGGATTCAT...CCT sequence2
marm
SingleLetterAlphabet() alignment with 2 rows a

In [217]:
same_gc_introns

{('chimp', 'Y'): [[121000, 176500, 202000, 267000, 289000],
  [122000, 177500, 203000, 268000, 290000]],
 ('gorilla',
  'Y'): [[138500,
   195000,
   220500,
   285500,
   307500,
   384500,
   500000,
   500500], [139500, 196000, 221500, 286500, 308500, 385500, 501000, 501500]],
 ('orangutan', 'Y'): [[500, 1500, 98500, 155000, 174000, 252500, 329000],
  [1500, 2500, 99500, 156000, 175000, 253500, 330000]],
 ('pileatedgibbon',
  'Y'): [[72000,
   112500,
   113000,
   145500,
   177000,
   177500,
   199000,
   264000,
   271500,
   324500,
   427000], [73000,
   113500,
   114000,
   146500,
   178000,
   178500,
   200000,
   265000,
   272500,
   325500,
   428000]],
 ('marm',
  'Y'): [[23000,
   23500,
   24000,
   111000,
   123000,
   123500,
   124000,
   140000,
   140500,
   148500,
   149000,
   165000,
   166500,
   182500,
   192500,
   213000,
   213500,
   214000], [24000,
   24500,
   25000,
   112000,
   124000,
   124500,
   125000,
   141000,
   141500,
   149500,
   

In [218]:
for key in same_gc_introns.keys():
    seq1= ""
    seq2 = ""
    sample_seqX = AlignIO.read('all_species/' + key[0] + '_intron_msa_' + key[1] + '.phy', "phylip")

    
    for ix in range(0, len(same_gc_introns[key][0])):
        s = same_gc_introns[key][0][ix]
        e = same_gc_introns[key][1][ix] #end list
       # print(s,e)
        new_seq = [seq[s:e] for seq in sample_seqX]
        seq1 = seq1+str(new_seq[0].seq)
        seq2 = seq2+str(new_seq[1].seq)

    alignment = MultipleSeqAlignment([])

# Create individual sequences and add them to the alignment
    record1 = seq1
    record2 = seq2

    newmsa = MultipleSeqAlignment([])


    new_record1 = SeqRecord.SeqRecord("")
    new_record1.id = "human"
    new_record1.seq = Seq(record1) #added this 
    newmsa.append(new_record1)
    new_record = SeqRecord.SeqRecord("")
    new_record.id = "sequence2"
    new_record.seq = Seq(record2)
    newmsa.append(new_record)
    
    AlignIO.write(newmsa, "all_species/" + key[0] + "_intron_msa_GCadj" + key[1] + ".phy", "phylip")


In [219]:
GC(newmsa[0].seq)

51.75

## Section 2: edit code so that it works per gene - can pull out intron from each species/gene pair

In [110]:
def give_me_gc_content_dict(type_of_seq):
    #type of seq = promoter or exon character
    species= ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'marm', 'mac']
    genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

    gc_prom_dict = {}

    for spec in species: 
        for gene in genes: 
            try:
                sample_seqY = AlignIO.read(gene + '/' + 'humanmasked.' + spec + '_' + type_of_seq + '_msa_filtered.phy', "phylip")
                gc_prom_dict[(spec, gene)] = GC(sample_seqY[0].seq)
            except: 
                next
    return gc_prom_dict
                                
gc_prom_dict =     give_me_gc_content_dict('promoter')
print(gc_prom_dict)
gc_exon_dict =     give_me_gc_content_dict('exon')
print(gc_exon_dict)


{('chimp', 'EIF1AX'): 61.91489361702128, ('chimp', 'EIF1AY'): 49.89429175475687, ('chimp', 'KDM5D'): 47.2, ('chimp', 'KDM5C'): 57.0, ('chimp', 'UTY'): 54.0, ('chimp', 'KDM6A'): 70.44967880085653, ('chimp', 'ZFY'): 68.78980891719745, ('chimp', 'ZFX'): 77.70114942528735, ('chimp', 'DDX3Y'): 47.37945492662474, ('chimp', 'DDX3X'): 57.66738660907127, ('chimp', 'USP9Y'): 54.02542372881356, ('chimp', 'USP9X'): 64.85788113695091, ('chimp', 'RPS4Y1'): 40.84507042253521, ('chimp', 'RPS4X'): 55.8, ('gorilla', 'EIF1AX'): 61.91489361702128, ('gorilla', 'EIF1AY'): 49.89429175475687, ('gorilla', 'KDM5D'): 47.294589178356716, ('gorilla', 'KDM5C'): 57.0, ('gorilla', 'UTY'): 54.0, ('gorilla', 'KDM6A'): 70.44967880085653, ('gorilla', 'ZFY'): 68.93617021276596, ('gorilla', 'ZFX'): 76.86746987951807, ('gorilla', 'DDX3Y'): 47.48953974895397, ('gorilla', 'DDX3X'): 57.66738660907127, ('gorilla', 'USP9Y'): 54.140127388535035, ('gorilla', 'USP9X'): 64.85788113695091, ('gorilla', 'RPS4Y1'): 40.6813627254509, ('g

In [111]:

def give_me_intron_coords(gc_prom_dict):
    
    same_gc_introns = {} #[[1,2,3], [4,5,6]] #starts list and end list
                     #can make these files have the exact same GC content 
 
    species= ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'marm', 'mac']

    genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 
    #genes = ["USP9X"] 

    for gene in genes: #only running through sample_seqY 

        for spec in species:
            try:
                sample = AlignIO.read(gene + '/' + 'humanmasked.' + spec + '_intron_msa_filtered.phy', "phylip")
                if len(sample) == 1: 
                    print('here', gene, spec)

                    continue 
            except: 
                continue 
                print(gene, spec)
            try:
                target = gc_prom_dict[(spec, gene)]
            except: 
                print(target) 
                continue
            
            range_gc = [target - 5, target + 5]
        
        #print(range_gc)
        
            for s in range(0, len(sample[0].seq), 50):                                                   #take thousand intervals that are +/- 5% of the promoter GC
                e = s + 50
                gc_perc = GC(sample[0][s:e].seq)
          #  print(range_gc)
                if (range_gc[0] < gc_perc) & (gc_perc < range_gc[1]):
               # print('yes', spec, gene)
                    try:
                        same_gc_introns[(spec,gene)][0].append(s)
                        same_gc_introns[(spec,gene)][1].append(e)
                    except: 
                        same_gc_introns[(spec,gene)] = [[s],[e]]
                    
    return same_gc_introns

same_gc_introns_promoters = give_me_intron_coords(gc_prom_dict)
same_gc_introns_exons = give_me_intron_coords(gc_exon_dict)


In [112]:
same_gc_introns_exons

{('chimp',
  'EIF1AX'): [[400,
   450,
   550,
   600,
   750,
   800,
   900,
   1050,
   1150,
   1300,
   1350,
   1500,
   1600,
   1650,
   1700,
   1950,
   2000,
   2050,
   2100,
   2400,
   2450,
   2650,
   2850,
   3000,
   3250,
   3300,
   3350,
   3600,
   3700,
   3850,
   4000,
   4250,
   4350,
   4450,
   4650,
   4750,
   4900,
   4950,
   5250,
   5650,
   5700,
   5850,
   6450,
   6600,
   6700,
   6750,
   6850,
   7150,
   7250,
   7300,
   7350,
   7400,
   7500,
   7600,
   7700,
   7800,
   7850,
   8000,
   8250,
   8950,
   9000,
   9150,
   9200,
   9300,
   9600,
   9700,
   9750,
   10200,
   10350,
   10600,
   10700,
   10800,
   10850], [450,
   500,
   600,
   650,
   800,
   850,
   950,
   1100,
   1200,
   1350,
   1400,
   1550,
   1650,
   1700,
   1750,
   2000,
   2050,
   2100,
   2150,
   2450,
   2500,
   2700,
   2900,
   3050,
   3300,
   3350,
   3400,
   3650,
   3750,
   3900,
   4050,
   4300,
   4400,
   4500,
   4700,
   4800,
   49

In [113]:
def make_new_intron_files(same_gc_introns, type_of_norm):

    for key, values in same_gc_introns.items():
        gene = key[1]
        spec = key[0]
        print(gene, spec)
        seq1= ""
        seq2 = ""

        sample = AlignIO.read(gene + '/' + 'humanmasked.' + spec + '_intron_msa_filtered.phy', "phylip")

        for ix in range(0, len(values[0])):
            s = same_gc_introns[key][0][ix]
            e = same_gc_introns[key][1][ix] #end list
           # print(s,e)
            new_seq = [seq[s:e] for seq in sample]
            seq1 = seq1+str(new_seq[0].seq)
            seq2 = seq2+str(new_seq[1].seq)

        alignment = MultipleSeqAlignment([])

    # Create individual sequences and add them to the alignment 
        record1 = seq1
        record2 = seq2

        newmsa = MultipleSeqAlignment([])

        new_record1 = SeqRecord.SeqRecord("")
        new_record1.id = "human"
        new_record1.seq = Seq(record1) #added this 
        newmsa.append(new_record1)
        new_record = SeqRecord.SeqRecord("")
        new_record.id = "sequence2"
        new_record.seq = Seq(record2)
        newmsa.append(new_record)
        print(len(record1))
        AlignIO.write(newmsa, gene + "/humanmasked." + spec + "_intron_msa_GCadj" + type_of_norm + ".phy", "phylip")
        
make_new_intron_files(same_gc_introns_exons, 'for_exon')
make_new_intron_files(same_gc_introns_promoters, 'for_promoter')


EIF1AX chimp
3650
EIF1AX gorilla
4050
EIF1AX orangutan
3150
EIF1AX pileatedgibbon
3050
EIF1AX marm
3800
EIF1AX mac
4000
EIF1AY chimp
5058
EIF1AY gorilla
5566
EIF1AY orangutan
4950
EIF1AY pileatedgibbon
4950
EIF1AY marm
4300
EIF1AY mac
5000
KDM5D chimp
2200
KDM5D gorilla
2500
KDM5D orangutan
2250
KDM5D pileatedgibbon
2650
KDM5D marm
1425
KDM5D mac
1950
KDM5C chimp
4400
KDM5C gorilla
5150
KDM5C orangutan
4850
KDM5C pileatedgibbon
5100
KDM5C marm
3627
KDM5C mac
4389
UTY chimp
58250
UTY gorilla
55650
UTY orangutan
52955
UTY pileatedgibbon
35000
UTY marm
31434
UTY mac
40350
KDM6A chimp
52590
KDM6A gorilla
53850
KDM6A orangutan
52240
KDM6A pileatedgibbon
48100
KDM6A marm
41485
KDM6A mac
50250
ZFY chimp
6150
ZFY gorilla
6600
ZFY orangutan
6150
ZFY pileatedgibbon
6450
ZFY mac
5500
ZFX chimp
8950
ZFX gorilla
8850
ZFX orangutan
9250
ZFX pileatedgibbon
8900
ZFX marm
7850
ZFX mac
9000
DDX3Y chimp
1534
DDX3Y gorilla
1550
DDX3Y orangutan
1366
DDX3Y pileatedgibbon
1500
DDX3Y marm
1400
DDX3Y mac
1383


In [117]:
gene = 'KDM5C'
spec = "mac"

#newmsa = AlignIO.read(gene + "/humanmasked." + spec + "_intron_msa_GCadj" + ".phy", "phylip")
newmsa = AlignIO.read(gene + "/humanmasked." + spec + "_intron_msa_GCadj" + ".phy", "phylip")
newmsa1 = AlignIO.read(gene + "/humanmasked." + spec + "_promoter_msa_filtered" + ".phy", "phylip")


print(GC(newmsa1[0].seq))
print(GC(newmsa[0].seq))

#print(gc_prom_dict[(spec, gene)])

57.11422845691383
53.63779527559055


In [115]:
same_gc_introns_exons.keys()

dict_keys([('chimp', 'EIF1AX'), ('gorilla', 'EIF1AX'), ('orangutan', 'EIF1AX'), ('pileatedgibbon', 'EIF1AX'), ('marm', 'EIF1AX'), ('mac', 'EIF1AX'), ('chimp', 'EIF1AY'), ('gorilla', 'EIF1AY'), ('orangutan', 'EIF1AY'), ('pileatedgibbon', 'EIF1AY'), ('marm', 'EIF1AY'), ('mac', 'EIF1AY'), ('chimp', 'KDM5D'), ('gorilla', 'KDM5D'), ('orangutan', 'KDM5D'), ('pileatedgibbon', 'KDM5D'), ('marm', 'KDM5D'), ('mac', 'KDM5D'), ('chimp', 'KDM5C'), ('gorilla', 'KDM5C'), ('orangutan', 'KDM5C'), ('pileatedgibbon', 'KDM5C'), ('marm', 'KDM5C'), ('mac', 'KDM5C'), ('chimp', 'UTY'), ('gorilla', 'UTY'), ('orangutan', 'UTY'), ('pileatedgibbon', 'UTY'), ('marm', 'UTY'), ('mac', 'UTY'), ('chimp', 'KDM6A'), ('gorilla', 'KDM6A'), ('orangutan', 'KDM6A'), ('pileatedgibbon', 'KDM6A'), ('marm', 'KDM6A'), ('mac', 'KDM6A'), ('chimp', 'ZFY'), ('gorilla', 'ZFY'), ('orangutan', 'ZFY'), ('pileatedgibbon', 'ZFY'), ('mac', 'ZFY'), ('chimp', 'ZFX'), ('gorilla', 'ZFX'), ('orangutan', 'ZFX'), ('pileatedgibbon', 'ZFX'), ('marm'

In [107]:
spec = "chimp"

#newmsa = AlignIO.read(gene + "/humanmasked." + spec + "_intron_msa_GCadj" + ".phy", "phylip")
newmsa = AlignIO.read('all_species/'  + spec + "_intron_msa_GCadjfor_promoter" + "_X" + ".phy", "phylip")
newmsa1 = AlignIO.read('all_species/'  + spec + "_promoter_msa_filtered" + "_Y" + ".phy", "phylip")

print(GC(newmsa[0].seq))
print(GC(newmsa1[0].seq))

#print(gc_prom_dict[(spec, gene)])

48.23529411764706
53.11837081035214
